In [1]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from pipeline_utils import (
    COMPUTE, CONCURRENT_TRIALS, DELAY_EVALUATION, EVALUATION_INTERVAL, GOAL, METRIC, SAMPLING_ALGORITHM, TIMEOUT, TIMEOUT_PLUS, TOTAL_TRIALS, #SWEEP
    SUBSCRIPTION, RESOURCE_GROUP, WS_NAME,  #AUTHENTICATE
    GBC_PATH, NBC_PATH, RFC_PATH, SVC_PATH, XGB_PATH, MULT_NBC_PATH,    #COMPONENTS PATHS
    INFOGAIN_PATH,    #FEAT SEL PATHS
    PREP_DATA_PATH,  #PREP DATA PATH
    GBC_BY_INFOGAIN, NBC_BY_INFOGAIN, RFC_BY_INFOGAIN, SVC_BY_INFOGAIN, XGB_BY_INFOGAIN, MULT_NBC_BY_INFOGAIN,   #PIPELINES
    INFOGAIN,   #FEAT SEL
    N_FEATURES, TRAIN_DATA, TEST_DATA, DATA_VERSION, #PIPELINE VALUES
    get_experiment_names,   #FUNCTIONS
)

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [2]:
data_to_train = ml_client.data.get(name=TRAIN_DATA.split('.')[0], version=DATA_VERSION)
print(f"Data to train asset URI: {data_to_train.path} - name: {TRAIN_DATA.split('.')[0]}")

data_to_test = ml_client.data.get(name=TEST_DATA.split('.')[0], version=DATA_VERSION)
print(f"Data to test asset URI: {data_to_test.path} - name: {TEST_DATA.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/da6ec459-95c4-4f18-8440-d275df8d38b7/resourcegroups/tcc-exp-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/515020ed3458cc913992fac6f6db5305/vrex_encoded_tf_idf_updated_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_.csv - name: vrex_encoded_tf_idf_updated_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_
Data to test asset URI: azureml://subscriptions/da6ec459-95c4-4f18-8440-d275df8d38b7/resourcegroups/tcc-exp-rg/workspaces/tcc-experiments/datastores/workspaceblobstore/paths/LocalUpload/12459a97b633042546916fa18a0378e1/vrex_encoded_tf_idf_updated_2018_2019_2020_2021_.csv - name: vrex_encoded_tf_idf_updated_2018_2019_2020_2021_


In [3]:
# importing the Component Package
from azure.ai.ml import load_component

data_prep_component = load_component(source=PREP_DATA_PATH)
data_prep_component = ml_client.create_or_update(data_prep_component)

feat_sel_component = load_component(source=INFOGAIN_PATH)
feat_sel_component = ml_client.create_or_update(feat_sel_component)

train_gbc = load_component(source=GBC_PATH)
train_gbc = ml_client.create_or_update(train_gbc)

train_nbc = load_component(source=NBC_PATH)
train_nbc = ml_client.create_or_update(train_nbc)

train_rfc = load_component(source=RFC_PATH)
train_rfc = ml_client.create_or_update(train_rfc)

train_svc = load_component(source=SVC_PATH)
train_svc = ml_client.create_or_update(train_svc)

train_xgb = load_component(source=XGB_PATH)
train_xgb = ml_client.create_or_update(train_xgb)

mult_nbc_train = load_component(source=MULT_NBC_PATH)
mult_nbc_train = ml_client.create_or_update(mult_nbc_train)


In [4]:

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

In [5]:
@dsl.pipeline(
    name=GBC_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_gbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_gbc_job = train_gbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        n_estimators_to_gbc=Choice(values=[50, 100, 200]),
        learning_rate_to_gbc=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_gbc = train_gbc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_gbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)

    sweep_step_to_gbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [6]:
@dsl.pipeline(
    name=NBC_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_nbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_nbc_job = train_nbc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
    )

In [7]:
@dsl.pipeline(
    name=RFC_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_rfc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_rfc_job = train_rfc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel, 
        n_estimators_to_rfc=Choice(values=[50, 100, 200]),
    )

    sweep_step_to_rfc = train_rfc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_rfc.set_limits(max_total_trials=TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT)
    sweep_step_to_rfc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [8]:
@dsl.pipeline(
    name=SVC_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_svc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_svc_job = train_svc(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        kernel_to_svc=Choice(values=["linear", "rbf", "poly", "sigmoid","precomputed"]),
        gamma_to_svc=Choice(values=["scale", "auto"]),
    )

    sweep_step_to_svc = train_svc_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_svc.set_limits(max_total_trials=4*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=2*TIMEOUT_PLUS)
    sweep_step_to_svc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [9]:
@dsl.pipeline(
    name=XGB_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_xgb_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    train_xgb_job = train_xgb(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,   
        n_estimators_to_xgb=Choice(values=[100, 500, 1000]),
        learning_rate_to_xgb=Uniform(min_value=0.01, max_value=0.3),
    )

    sweep_step_to_xgb = train_xgb_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_xgb.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)
    sweep_step_to_xgb.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)

In [10]:
@dsl.pipeline(
    name=MULT_NBC_BY_INFOGAIN,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_mult_nbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    mult_nbc_train_job = mult_nbc_train(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        alpha=Choice(values=[0.01, 0.1, 0.5, 1.0, 2, 5]),
        fit_prior=Choice(values=[True, False]),
    )

    sweep_step_to_mult_nbc = mult_nbc_train_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_mult_nbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)

    sweep_step_to_mult_nbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [11]:
pipelines = []

for n_feature in N_FEATURES:
    pipelines.append(train_mult_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_nbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_gbc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_rfc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))
    
    pipelines.append(train_svc_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))

    pipelines.append(train_xgb_pipeline(
        data_to_train=Input(type="uri_file", path=data_to_train.path),
        data_to_test=Input(type="uri_file", path=data_to_test.path),
        feature_quantity=n_feature,
        flag_remove_null_values=False,
        flag_remove_values_by_percentage=False,
        percentage_to_remove_column=0,
    ))


In [12]:
experiment_names = get_experiment_names(INFOGAIN)
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )
    ml_client.jobs.stream(pipeline_job.name)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Infogain_MultGaussianNB_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_GaussianNB_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_GradientBoostingClassifier_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_RandomForestClassifier_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_SVC_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_XGBoost_N_FEAT_399_DATE_2024_07_16_04_06_59
Infogain_MultGaussianNB_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_GaussianNB_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_GradientBoostingClassifier_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_RandomForestClassifier_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_SVC_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_XGBoost_N_FEAT_300_DATE_2024_07_16_04_06_59
Infogain_MultGaussianNB_N_FEAT_200_DATE_2024_07_16_04_06_59
Infogain_GaussianNB_N_FEAT_200_DATE_2024_07_16_04_06_59
Infogain_GradientBoostingClassifier_N_FEAT_200_DATE_2024_07_16_04_06_59
Infogain_RandomForestClassifier_N_FEAT_200_DATE_2024_07_16_04_06_59
Infogain_SVC_N_FEAT_200_DATE